# Oil Project
### Market Analysis by Countries

In [6]:
import sqlalchemy as sqlalchemy_package
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.declarative import DeferredReflection
from sqlalchemy import *
import pandas as pd
import gmaps
import warnings
import config as creds
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from matplotlib.cm import viridis, plasma
from my_func import GoogleMapClient, GmapsAnalysis

sns.set()
warnings.filterwarnings('ignore')
plt.rc('lines', linewidth=1)
rcParams['figure.figsize'] = 12,6
pd.set_option('display.max_colwidth', 40)
pd.options.display.float_format = '{:,.2f}'.format

In [20]:
# Set up a connection to the postgres server
class postgre_sql():
    def __init__(self, dbase = creds.PGDATABASE, dbschema = 'countries'):
        DATABASE_URL = f"postgres://{creds.PGUSER}:{creds.PGPASSWORD}@{creds.PGHOST}:5432/{dbase}"
        engine = sqlalchemy_package.create_engine(DATABASE_URL,
                connect_args={'options': '-csearch_path={}'.format(dbschema)})
        self.engine = engine
    def auto_map(self):
        # Declare a Base with explicit schema using `automap_base()`
        Base = automap_base(bind=self.engine)
        # Use the Base class to reflect the database tables
        Base.prepare(self.engine, reflect=True)
        return Base
    def read_df(self, table='countries'):
        return pd.read_sql(table, self.engine)

In [22]:
# Print all of the classes mapped to the Base
countries_db = postgre_sql()
base = countries_db.auto_map()
base.classes.keys()

['countries', 'production']

In [24]:
sql_product = base.classes.production
# Create a session
session = Session(countries_db.engine)
# Use the session to query Demographics table and display the first 5 locations
stmt = session.query(sql_product.country, func.avg(sql_product.oil_production).label('2019')).\
                    filter(sql_product.year == 2020).group_by(sql_product.country).\
                    order_by(desc('2019')).statement
data = pd.read_sql_query(stmt, session.bind)
data.head(10)

,country,2019
0,Saudi Arabia,"12,328.50"
1,Russia,"11,509.50"
2,United States of America,"7,545.20"
3,Canada,"5,382.50"
4,Iraq,"4,615.00"
5,United Arab Emirates,"4,315.00"
6,Brazil,"3,445.00"
7,Kuwait,"3,082.50"
8,China,"3,073.67"
9,Iran,"2,902.00"


In [97]:
oil_map = GmapsAnalysis(data)
oil_map.color_map(cmap_color = plasma)
oil_map.fig

Figure(layout=FigureLayout(height='420px'))

![oil_map](images/oil_prod_2019.png)